Procesamiento de Datos y Consolidación de la Base de Datos Climática
Una vez obtenidos los enlaces directos a los archivos históricos, este notebook ejecuta la fase crítica de ETL (Extracción, Transformación y Carga). El desafío principal aquí radica en la heterogeneidad y falta de estructura de los archivos .txt de CONAGUA, los cuales están diseñados para lectura humana y no para procesamiento computacional.

La estrategia de consolidación se basa en los siguientes pilares técnicos:

1. Extracción Masiva Multihilo
Dado que el proyecto gestiona más de 5,300 estaciones, realizar peticiones secuenciales sería ineficiente. El script implementa un flujo de procesamiento masivo que:

- Realiza peticiones HTTP a cada URL obtenida en la fase anterior.

- Valida la disponibilidad del archivo antes de intentar el parseo.

- Gestiona errores de conexión para asegurar que una URL caída no interrumpa el flujo total.

2. Ingeniería de Parseo de Texto Plano
Los archivos .txt contienen tablas climatológicas con formatos variables. Para estandarizar esta información, el código realiza las siguientes operaciones:

- Identificación de Patrones: Localiza las secciones de interés (Precipitación Total o Temperatura Media) dentro del cuerpo del texto.

- Limpieza de Ruido: Elimina encabezados, pies de página y caracteres especiales que no aportan valor numérico.

- Manejo de Valores Nulos: Detecta y estandariza los códigos de "datos faltantes" propios de las estaciones meteorológicas para evitar sesgos en el análisis posterior.

3. Estructuración y Normalización de la Base de Datos
- El resultado final es la creación de un DataFrame maestro con las siguientes características:

- Indexación por Estación y Tiempo: Cada registro se vincula inequívocamente a su ID de estación (Estacion/id) y su correspondiente año.

- Transformación de Columnas (Pivotado): Los datos mensuales se organizan en columnas estandarizadas (Enero a Diciembre), facilitando el cálculo de promedios anuales y tendencias estacionales.

- Tipado de Datos: Se asegura que todas las métricas climáticas sean de tipo flotante (float) para permitir operaciones aritméticas inmediatas.

4. Exportación y Persistencia
Para garantizar la portabilidad de la información, el notebook culmina con la exportación de los datos consolidados a formato CSV.

- Encoding UTF-8: Se utiliza codificación estándar para preservar la integridad de los nombres de estaciones con caracteres especiales o acentos.

- Auditoría de Carga: El sistema imprime un resumen final indicando el número total de filas procesadas y los tipos de datos asignados, sirviendo como un check de calidad de la base de datos resultante.

- Impacto del Proyecto: Esta base de datos consolidada reduce semanas de trabajo manual a solo minutos de ejecución automatizada, permitiendo que investigadores y analistas se enfoquen en la interpretación del clima en lugar de la limpieza de archivos.

- Ejemplo visual de la informacion sin estructurar

![alt text](image-1.png)

In [ ]:
#Extraccion masiva de LLUVIA TOTAL MENSUAL desde URLs txt de CONAGUA
import pandas as pd
import requests
from io import StringIO
import re
import os
import time
import random

# --- CONFIGURACIÓN ---
# Ruta donde se encuentra el listado de URLs
RUTA_ARCHIVO_URLS = r"resultados\URL - Listado limpio.csv"
COLUMNA_URL = "URL_Mensual" 


# --- FUNCIÓN DE EXTRACCIÓN POR URL (CON CONVERSIÓN DE CLAVES A STRING) ---

def crear_df_lluvia_mensual(url: str) -> pd.DataFrame:
    """
    Descarga el archivo TXT de la CONAGUA y procesa EXCLUSIVAMENTE
    la sección 'LLUVIA TOTAL MENSUAL'. Convierte las claves 'AÑO' y 
    'Estacion/id' a string.
    """
    SECCION_CLAVE = "LLUVIA TOTAL MENSUAL"
    
    # 1. Descargar el contenido del archivo
    try:
        user_agents = [
         """
           Se recomienta el uso de Mozila
                   """
        ]
        headers = {'User-Agent': random.choice(user_agents)}

        response = requests.get(url, headers=headers, timeout=30) 
        response.raise_for_status() 
        contenido = response.text
    except requests.exceptions.RequestException:
        return pd.DataFrame()

    # 2. Extraer Metadatos (Encabezado)
    patrones = {
        'Estacion/id': r"ESTACIÓN\s*:\s*(\d+)",
        'Nombre': r"NOMBRE\s*:\s*([^\n]+)",
        'Estado': r"ESTADO\s*:\s*([^\n]+)",
        'Municipio': r"MUNICIPIO\s*:\s*([^\n]+)",
        'Situación': r"SITUACIÓN\s*:\s*([^\n]+)",
        'Latitude': r"LATITUD\s*:\s*([\d\.-]+\s*°)",
        'Longitude': r"LONGITUD\s*:\s*([\d\.-]+\s*°)",
    }
    
    metadata = {}
    for key, pattern in patrones.items():
        match = re.search(pattern, contenido)
        if match:
            value = match.group(1).strip().replace(' °', '').replace('°', '')
            metadata[key] = value

    # 3. AISLAR Y LIMPIAR SOLO la SECCION_CLAVE
    inicio_seccion = contenido.find(SECCION_CLAVE)
    
    if inicio_seccion == -1:
        # MANEJO DE ERROR: Advertir si la sección no existe
        estacion_id = metadata.get('Estacion/id', 'N/A')
        print(f"   [ADVERTENCIA] Sección '{SECCION_CLAVE}' NO ENCONTRADA en URL: {url}. Estación ID: {estacion_id}")
        return pd.DataFrame()
    
    seccion_completa = contenido[inicio_seccion:]
    datos_tabla_limpio = []
    capturar_datos = False
    
    for linea in seccion_completa.splitlines():
        linea = linea.strip()
        
        if SECCION_CLAVE in linea:
            capturar_datos = True 
            continue
        
        # Detener al encontrar filas estadísticas (FIN DE LA TABLA)
        if "MÍNIMA" in linea or "MÁXIMA" in linea or "MEDIA" in linea or "DESV.ST" in linea:
            break
            
        if capturar_datos and linea:
            datos_tabla_limpio.append(linea)

    # 4. Cargar los datos en un DataFrame
    datos_df = "\n".join(datos_tabla_limpio)
    
    if not datos_df.strip():
        return pd.DataFrame()
        
    try:
        # Lectura de la tabla (TSV)
        df = pd.read_csv(StringIO(datos_df), sep='\t', skipinitialspace=True, na_values=['', ' '], engine='python')
    except Exception:
        # Falla al convertir la tabla a DataFrame
        return pd.DataFrame()


    # 5. Aplicar la limpieza y la estructura final solicitada
    columnas_renombrar = {
        'ACUM': 'Acum Σ', 'PROM': 'Promedio', 'MESES': 'Meses',
        'ENE': 'Enero', 'FEB': 'Febrero', 'MAR': 'Marzo', 'ABR': 'Abril', 'MAY': 'Mayo',
        'JUN': 'Junio', 'JUL': 'Julio', 'AGO': 'Agosto', 'SEP': 'Septiembre', 
        'OCT': 'Octubre', 'NOV': 'Noviembre', 'DIC': 'Diciembre' 
    }
    df.rename(columns=columnas_renombrar, inplace=True)

    # Convertir AÑO a string antes de insertar metadatos
    if 'AÑO' in df.columns:
        df['AÑO'] = df['AÑO'].astype(str).str.strip()
        
    # Insertar columnas de metadatos y reordenar
    for i, (key, value) in enumerate(metadata.items()):
        # CONVERSIÓN CLAVE: Insertar metadatos como string
        df.insert(loc=i, column=key, value=str(value)) 
        
    # CONVERSIÓN CLAVE FINAL: Asegurar que 'Estacion/id' (metadato) sea string
    if 'Estacion/id' in df.columns:
        df['Estacion/id'] = df['Estacion/id'].astype(str).str.strip()


    columnas_meses_completos = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 
                                'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 
                                'Diciembre']
                      
    columnas_finales = list(metadata.keys()) + ['AÑO'] + columnas_meses_completos + ['Acum Σ', 'Promedio', 'Meses']
    df = df.reindex(columns=columnas_finales)
    
    return df

# --- FUNCIÓN DE PROCESAMIENTO MASIVO COMPLETO ---

def procesar_extraccion_masiva(ruta_csv_urls: str, columna_url: str) -> pd.DataFrame:
    """
    Coordina la descarga y procesamiento de TODAS las URLs, consolidando los resultados.
    """
    if not os.path.exists(ruta_csv_urls):
        print(f"Error de archivo: No se encontró el archivo de URLs en la ruta: {ruta_csv_urls}")
        return pd.DataFrame()

    try:
        df_urls = pd.read_csv(ruta_csv_urls, usecols=[columna_url])
        urls_a_procesar = df_urls[columna_url].dropna().unique().tolist()
    except Exception as e:
        print(f"Error al cargar URLs desde el CSV: {e}")
        return pd.DataFrame()

    if not urls_a_procesar:
        print("Advertencia: No se encontraron URLs válidas para procesar.")
        return pd.DataFrame()

    total_urls = len(urls_a_procesar)
    print(f"\nIniciando extracción masiva completa de {total_urls} URLs para LLUVIA TOTAL MENSUAL...")
    
    lista_df_estaciones = []
    
    for i, url in enumerate(urls_a_procesar):
        print(f"-> Procesando {i+1} de {total_urls}: {url}")
        
        try:
            df_estacion = crear_df_lluvia_mensual(url)
            
            if not df_estacion.empty:
                lista_df_estaciones.append(df_estacion)
                print(f"   [ÉXITO] Estación procesada.")
                
        except Exception as e:
            print(f"   [ERROR INESPERADO] Falló el procesamiento interno de {url}: {e}")
            
        # Pausa aleatoria para no saturar el servidor
        time.sleep(random.uniform(1, 3)) 
            
    if not lista_df_estaciones:
        print("\n¡PROCESO FINALIZADO! No se pudieron extraer datos válidos para consolidar.")
        return pd.DataFrame()
        
    print(f"\nConsolidando {len(lista_df_estaciones)} DataFrames...")
    df_consolidado = pd.concat(lista_df_estaciones, ignore_index=True)
    
    print(f"Total de filas consolidadas en la variable: {df_consolidado.shape[0]}")
    return df_consolidado

# --- BLOQUE DE EJECUCIÓN ---

# 1. Ejecutar la extracción masiva completa
df_base_datos_lluvia = procesar_extraccion_masiva(
    ruta_csv_urls=RUTA_ARCHIVO_URLS, 
    columna_url=COLUMNA_URL
)

print("\n--- PROCESO DE EXTRACCIÓN MASIVA FINALIZADO ---")

# 2. El DataFrame consolidado se encuentra en la variable 'df_base_datos_lluvia'
if not df_base_datos_lluvia.empty:
    print(f"\nBase de datos consolidada (LLUVIA TOTAL MENSUAL) creada en la variable 'df_base_datos_lluvia'.")
    print(f"Tipos de dato de las columnas clave: 'Estacion/id' ({df_base_datos_lluvia['Estacion/id'].dtype}) y 'AÑO' ({df_base_datos_lluvia['AÑO'].dtype})")
    print(f"Primeras 5 filas del resultado (Total de Filas: {df_base_datos_lluvia.shape[0]}):")
    print(df_base_datos_lluvia.head())
else:
    print("\nLa variable 'df_base_datos_lluvia' está vacía. No se pudo extraer información válida.")

In [ ]:
df_base_datos_lluvia.to_csv(r"resultados\Base de datos Lluvia.csv", index=False)

In [ ]:
#Extraccion masiva de TEMPERATURA MEDIA MENSUAL desde URLs txt de CONAGUA
import pandas as pd
import requests
from io import StringIO
import re
import os
import time
import random

# --- CONFIGURACIÓN ---
# Ruta donde se encuentra el listado de URLs (misma que la usada para la lluvia)
RUTA_ARCHIVO_URLS = r"resultados\RL - Listado limpio.csv"
COLUMNA_URL = "URL_Mensual" 


# --- FUNCIÓN DE EXTRACCIÓN POR URL (TEMPERATURA) ---

def crear_df_temperatura_mensual(url: str) -> pd.DataFrame:
    """
    Descarga el archivo TXT de la CONAGUA, extrae metadatos y procesa 
    EXCLUSIVAMENTE la sección 'TEMPERATURA MEDIA MENSUAL'.
    Convierte las claves 'AÑO' y 'Estacion/id' a string.
    """
    SECCION_CLAVE = "TEMPERATURA MEDIA MENSUAL"
    
    # 1. Descargar el contenido del archivo
    try:
        # Usamos un User-Agent y un timeout para una descarga más robusta
        user_agents = [
           "se recomienda el uso de " 
        ]
        headers = {'User-Agent': random.choice(user_agents)}

        response = requests.get(url, headers=headers, timeout=30) 
        response.raise_for_status() 
        contenido = response.text
    except requests.exceptions.RequestException:
        return pd.DataFrame()

    # 2. Extraer Metadatos (Encabezado)
    patrones = {
        'Estacion/id': r"ESTACIÓN\s*:\s*(\d+)",
        'Nombre': r"NOMBRE\s*:\s*([^\n]+)",
        'Estado': r"ESTADO\s*:\s*([^\n]+)",
        'Municipio': r"MUNICIPIO\s*:\s*([^\n]+)",
        'Situación': r"SITUACIÓN\s*:\s*([^\n]+)",
        'Latitude': r"LATITUD\s*:\s*([\d\.-]+\s*°)",
        'Longitude': r"LONGITUD\s*:\s*([\d\.-]+\s*°)",
    }
    
    metadata = {}
    for key, pattern in patrones.items():
        match = re.search(pattern, contenido)
        if match:
            value = match.group(1).strip().replace(' °', '').replace('°', '')
            metadata[key] = value

    # 3. AISLAR Y LIMPIAR SOLO la SECCION_CLAVE
    inicio_seccion = contenido.find(SECCION_CLAVE)
    
    if inicio_seccion == -1:
        # MANEJO DE ERROR SOLICITADO: Advertir si la sección no existe
        estacion_id = metadata.get('Estacion/id', 'N/A')
        print(f"   [ADVERTENCIA] Sección '{SECCION_CLAVE}' NO ENCONTRADA en URL: {url}. Estación ID: {estacion_id}")
        return pd.DataFrame()
    
    seccion_completa = contenido[inicio_seccion:]
    datos_tabla_limpio = []
    capturar_datos = False
    
    for linea in seccion_completa.splitlines():
        linea = linea.strip()
        
        if SECCION_CLAVE in linea:
            capturar_datos = True 
            continue
        
        # Detener al encontrar filas estadísticas (FIN DE LA TABLA)
        if "MÍNIMA" in linea or "MÁXIMA" in linea or "MEDIA" in linea or "DESV.ST" in linea:
            break
            
        if capturar_datos and linea:
            datos_tabla_limpio.append(linea)

    # 4. Cargar los datos en un DataFrame
    datos_df = "\n".join(datos_tabla_limpio)
    
    if not datos_df.strip():
        return pd.DataFrame()
        
    try:
        df = pd.read_csv(StringIO(datos_df), sep='\t', skipinitialspace=True, na_values=['', ' '], engine='python')
    except Exception:
        return pd.DataFrame()


    # 5. Aplicar la limpieza, estructura y tipado
    columnas_renombrar = {
        # Mantengo 'Acum Σ' por consistencia con la estructura de la función original,
        # aunque en temperatura a menudo es irrelevante o se usa para Suma de T
        'ACUM': 'Acum T', 
        'PROM': 'Promedio',
        'MESES': 'Meses',
        'ENE': 'Enero', 'FEB': 'Febrero', 'MAR': 'Marzo', 'ABR': 'Abril', 'MAY': 'Mayo',
        'JUN': 'Junio', 'JUL': 'Julio', 'AGO': 'Agosto', 'SEP': 'Septiembre', 
        'OCT': 'Octubre', 'NOV': 'Noviembre', 'DIC': 'Diciembre' 
    }
    df.rename(columns=columnas_renombrar, inplace=True)

    # CONVERSIÓN A STRING: AÑO
    if 'AÑO' in df.columns:
        df['AÑO'] = df['AÑO'].astype(str).str.strip()
        
    # Insertar columnas de metadatos (claves se insertan como string)
    for i, (key, value) in enumerate(metadata.items()):
        df.insert(loc=i, column=key, value=str(value)) 
        
    # CONVERSIÓN A STRING: Estacion/id
    if 'Estacion/id' in df.columns:
        df['Estacion/id'] = df['Estacion/id'].astype(str).str.strip()


    columnas_meses_completos = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 
                                'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 
                                'Diciembre']
                      
    columnas_finales = list(metadata.keys()) + ['AÑO'] + columnas_meses_completos + ['Acum T', 'Promedio', 'Meses']
    df = df.reindex(columns=columnas_finales)
    
    return df

# --- FUNCIÓN DE PROCESAMIENTO MASIVO (TEMPERATURA) ---

def procesar_extraccion_masiva_temperatura(ruta_csv_urls: str, columna_url: str) -> pd.DataFrame:
    """
    Coordina la descarga y procesamiento de TODAS las URLs para Temperatura Media Mensual.
    """
    if not os.path.exists(ruta_csv_urls):
        print(f"Error de archivo: No se encontró el archivo de URLs en la ruta: {ruta_csv_urls}")
        return pd.DataFrame()

    try:
        df_urls = pd.read_csv(ruta_csv_urls, usecols=[columna_url])
        urls_a_procesar = df_urls[columna_url].dropna().unique().tolist()
    except Exception as e:
        print(f"Error al cargar URLs desde el CSV: {e}")
        return pd.DataFrame()

    if not urls_a_procesar:
        print("Advertencia: No se encontraron URLs válidas para procesar.")
        return pd.DataFrame()

    total_urls = len(urls_a_procesar)
    print(f"\nIniciando extracción masiva completa de {total_urls} URLs para TEMPERATURA MEDIA MENSUAL...")
    
    lista_df_estaciones = []
    
    for i, url in enumerate(urls_a_procesar):
        print(f"-> Procesando {i+1} de {total_urls}: {url}")
        
        try:
            # Llama a la función específica de temperatura
            df_estacion = crear_df_temperatura_mensual(url)
            
            if not df_estacion.empty:
                lista_df_estaciones.append(df_estacion)
                print(f"   [ÉXITO] Estación procesada.")
                
        except Exception as e:
            print(f"   [ERROR INESPERADO] Falló el procesamiento interno de {url}: {e}")
            
        # Pausa aleatoria
        time.sleep(random.uniform(1, 3)) 
            
    if not lista_df_estaciones:
        print("\n¡PROCESO FINALIZADO! No se pudieron extraer datos válidos para consolidar.")
        return pd.DataFrame()
        
    print(f"\nConsolidando {len(lista_df_estaciones)} DataFrames...")
    df_consolidado = pd.concat(lista_df_estaciones, ignore_index=True)
    
    print(f"Total de filas consolidadas en la variable: {df_consolidado.shape[0]}")
    return df_consolidado

# --- BLOQUE DE EJECUCIÓN ---

# 1. Ejecutar la extracción masiva completa para TEMPERATURA
df_base_datos_temperatura = procesar_extraccion_masiva_temperatura(
    ruta_csv_urls=RUTA_ARCHIVO_URLS, 
    columna_url=COLUMNA_URL
)

print("\n--- PROCESO DE EXTRACCIÓN DE TEMPERATURA FINALIZADO ---")

# 2. El DataFrame consolidado se encuentra en la variable 'df_base_datos_temperatura'
if not df_base_datos_temperatura.empty:
    print(f"\n✅ Base de datos consolidada (TEMPERATURA MEDIA MENSUAL) creada en la variable 'df_base_datos_temperatura'.")
    print(f"Tipos de dato de las columnas clave: 'Estacion/id' ({df_base_datos_temperatura['Estacion/id'].dtype}) y 'AÑO' ({df_base_datos_temperatura['AÑO'].dtype})")
    print(f"Primeras 5 filas del resultado (Total de Filas: {df_base_datos_temperatura.shape[0]}):")
    print(df_base_datos_temperatura.head())
else:
    print("\nLa variable 'df_base_datos_temperatura' está vacía. No se pudo extraer información válida.")

In [ ]:
df_base_datos_temperatura.to_csv(r"resultados\Base de datos temperatura.csv", index=False, encoding='utf-8')